In [1]:
# 📌 PC1~PC5에서 반복적으로 중요한 변수:
pca_cols = [
    'CA이자율_할인전', 'CL이자율_할인전', 'RV_평균잔액_R3M', 'RV일시불이자율_할인전', 'RV최소결제비율', 'RV현금서비스이자율_할인전', 
    '방문월수_앱_R6M', '방문일수_앱_B0M', '방문일수_앱_R6M', '방문횟수_앱_B0M', '방문후경과월_앱_R6M', 
    '이용금액_R3M_신용', '이용금액_R3M_신용체크', '이용금액_일시불_B0M', '이용금액대', 
    '일시불ONLY전환가능여부', 
    '잔액_리볼빙일시불이월_B0M', '잔액_일시불_B0M', '잔액_일시불_B1M', '잔액_일시불_B2M', '잔액_카드론_B0M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', 
    '정상청구원금_B0M', '정상청구원금_B2M', '정상청구원금_B5M', 
    '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '최종카드론_대출금액', '카드론이용금액_누적', '평잔_RV일시불_3M', '평잔_RV일시불_6M', '평잔_일시불_3M', '평잔_일시불_6M', 
    '평잔_카드론_3M', '평잔_카드론_6M', '평잔_할부_3M', '홈페이지_금융건수_R3M', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R3M', '홈페이지_선결제건수_R6M'
    ]

In [2]:
base_cols = ["ID","Segment"]

In [3]:
selected_cols = pca_cols + base_cols

In [4]:
# Modin 벡엔드를 ray로 강제 설정


# 이후 Modin 코드 실행
import modin.pandas as pd
df = pd.DataFrame({"a": [1,2,3]})
print(df)


# 파일 경로
file_path = "../../../data/통합_train_데이터.parquet"
df = pd.read_parquet(file_path)

   a
0  1
1  2
2  3


2025-07-10 14:11:07,765 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:53975 (pid=17420) exceeded 95% memory budget. Restarting...
2025-07-10 14:11:10,276 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:53971 (pid=18892) exceeded 95% memory budget. Restarting...
2025-07-10 14:11:10,416 - distributed.nanny - WARNING - Restarting worker
2025-07-10 14:11:10,684 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:53969 (pid=6272) exceeded 95% memory budget. Restarting...
2025-07-10 14:11:11,363 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:53969' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'parse-2b6f88ff69a64f01ed22761d6964c123', 'lambda-e96255760db5f9bd771cb86b70bd1eb2'} (stimulus_id='handle-worker-cleanup-1752124271.3626504')
2025-07-10 14:11:11,730 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:53967 (pid=18604) exceeded 95% memory budget. Restarting...
2025-07-10 

KilledWorker: Attempted to run task 'parse-f21ed0a8a83a60e0703ffc2cd176ce6c' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:54094. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

2025-07-10 14:13:05,057 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:54462' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'parse-5c8d1072a094f59a0b73bf2f870a5dc7'} (stimulus_id='handle-worker-cleanup-1752124385.056405')
2025-07-10 14:13:05,064 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:54395' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'lambda-bbc1b27b64c98b3c9c97a2650baf8e98', 'lambda-5f36c92bf874021df92202d3369baf42', 'lambda-3947224b284d23576fb31e830e213aa4', 'lambda-23aad8b835b7b772ddf567f7880413b4', 'lambda-d07a7b46d197e6ccc064770b29af67a9', 'lambda-cf61cabc2a40762108f101346a3d1ed7', 'lambda-6d230cb99c75e91d49852221d3e6b6a9', 'lambda-284c0114ef9b23b01851301e8613ed0f', 'lambda-8b51a00ed557f244acbd662fd178f37e', 'lambda-98fd10a58ba1722b3e4b849498bf69bc', 'parse-9f3fa5e57bee07e83e19da82dedd87bb', 'parse-4a2772953aa0b63b6ce9337f14548e8c', 'parse-7

In [ ]:
print(len(selected_cols))         
print(type(selected_cols[0]))

In [ ]:
def map_categorical_columns(df, verbose=True):
    """
    미리 정의된 매핑 기준에 따라 범주형 컬럼들을 수치형으로 변환합니다.
    처리 컬럼: 거주시도명, 연회비발생카드수_B0M, 한도증액횟수_R12M, 이용금액대,
              할인건수_R3M, 할인건수_B0M, 방문횟수_PC_R6M, 방문횟수_앱_R6M, 방문일수_PC_R6M
    """

    # 1. 거주시도명 → 수도권 여부
    capital_area = ['서울특별시', '경기도', '인천광역시']
    if '거주시도명' in df.columns:
        df['거주시도_수도권여부'] = df['거주시도명'].apply(lambda x: 1 if x in capital_area else 0)
        df.drop(columns=['거주시도명'], inplace=True)
        if verbose: print("[거주시도명] → 수도권 여부 인코딩 완료")

    # 2. 연회비발생카드수_B0M
    mapping = {"0개": 0, "1개이상": 1}
    if '연회비발생카드수_B0M' in df.columns:
        df['연회비발생카드수_B0M'] = df['연회비발생카드수_B0M'].map(mapping).astype(int)
        if verbose: print("[연회비발생카드수_B0M] 인코딩 완료")

    # 3. 한도증액횟수_R12M
    mapping = {"0회": 0, "1회이상": 1}
    if '한도증액횟수_R12M' in df.columns:
        df['한도증액횟수_R12M'] = df['한도증액횟수_R12M'].map(mapping).astype(int)
        if verbose: print("[한도증액횟수_R12M] 인코딩 완료")

    # 4. 이용금액대 (중간값 기준: 만원 단위)
    mapping = {
        "09.미사용": 0,
        "05.10만원-": 5,
        "04.10만원+": 20,
        "03.30만원+": 40,
        "02.50만원+": 75,
        "01.100만원+": 150
    }
    if '이용금액대' in df.columns:
        df['이용금액대'] = df['이용금액대'].map(mapping)
        if verbose: print("[이용금액대] 중간값 인코딩 완료")

    # 5. 할인건수 인코딩
    discount_map = {
        "1회 이상": 1,
        "10회 이상": 10,
        "20회 이상": 20,
        "30회 이상": 30,
        "40회 이상": 40
    }
    for col in ['할인건수_R3M', '할인건수_B0M']:
        if col in df.columns:
            df[col] = df[col].map(discount_map).astype(int)
            if verbose: print(f"[{col}] 인코딩 완료")

    # 6. 방문횟수 및 방문일수 인코딩
    visit_map = {
        "1회 이상": 1,
        "10회 이상": 10,
        "20회 이상": 20,
        "30회 이상": 30,
        "40회 이상": 40,
        "50회 이상": 50,
        "60회 이상": 60,
        "70회 이상": 70,
        "80회 이상": 80
    }

    visit_cols = ['방문횟수_PC_R6M', '방문횟수_앱_R6M', '방문일수_PC_R6M']
    for col in visit_cols:
        if col in df.columns:
            df[col] = df[col].map(visit_map).astype(int)
            if verbose: print(f"[{col}] 인코딩 완료")

    return df


In [ ]:
import modin.pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# 1. 데이터 불러오기
df = pd.read_parquet("../../../data/통합_train_데이터.parquet")

# 2. 피처 및 타겟 분리
X = df[pca_cols].copy() 
y = df["Segment"]

X = X.loc[:, ~X.columns.duplicated()] #중복제거

# 3. 범주형 인코딩
df = map_categorical_columns(df)
cat_cols = X.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# 4. 결측치 처리
X = pd.DataFrame(SimpleImputer(strategy='mean').fit_transform(X), columns=X.columns)

# 스케일링 (DataFrame 형태 유지)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 라벨인코딩
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)

# 6. train-validation 분할
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# CPU 모델 사용
xgb_model = XGBClassifier(
    tree_method='hist',         # GPU 대신 CPU 전용 히스토그램 기반
    predictor='auto',           # 자동 설정 (CPU에 맞게)
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)


# 8. 학습
xgb_model.fit(X_train, y_train)

# 9. 예측 및 평가
y_pred = xgb_model.predict(X_val)
print(classification_report(y_val, y_pred))


### 전환 후보군 대상이 되는 피처 탐색
- 예측 확률이 0.6 이상 되는 피처를 선택한다.

In [ ]:
import numpy as np
import modin.pandas as pd

# 10. 예측 확률 계산
y_proba = xgb_model.predict_proba(X_val)  # 클래스별 확률 반환

# 11. 가장 높은 확률의 클래스 선택
y_pred = np.argmax(y_proba, axis=1)

# 12. 확률이 0.6 이상인 고객만 추출
threshold = 0.6
high_confidence_mask = np.max(y_proba, axis=1) >= threshold

# 13. 결과 정리
result_df = X_val.copy()
result_df['real_segment'] = le_y.inverse_transform(y_val)
result_df['predicted_segment'] = le_y.inverse_transform(y_pred)
result_df['predicted_prob'] = np.max(y_proba, axis=1)

# 14. 확률이 0.6 이상인 전환 후보군만 추출
candidate_df = result_df[high_confidence_mask]

# 15. 상위 10개 미리보기
print("✅ 확률 0.6 이상인 전환 후보군:")
print(candidate_df[['real_segment', 'predicted_segment', 'predicted_prob']].head(10))

### 전환 경계선에 있는 고객
- 0.5 ~ 0.74 사이에 있는 고객을 전환 후보군으로 지정한다.

In [ ]:
# 예측 확률이 0.5 ~ 0.74 사이이면서, 실제와 예측이 다른 경우만!
unstable_candidates = result_df[
    (result_df['predicted_prob'] >= 0.5) &
    (result_df['predicted_prob'] <= 0.74) &
    (result_df['real_segment'] != result_df['predicted_segment'])
]

print(unstable_candidates[['real_segment', 'predicted_segment', 'predicted_prob']].head())

### 없다니??

#### 현재 조건
- 예측 확률이 0.5 이상 0.74 이하인 사람이면서
- 실제 세그먼트랑 예측 세그먼트가 다른 사람

즉!! 이 두 조건을 모두 만족하는 사람이 없다는 뜻

## 원인은?
### ✅ 왜 없을 수도 있냐?
#### 모델이 너무 확신하고 있어서
→ 대부분의 예측 확률이 0.9 이상이야 (실제로 출력된 거 보면 거의 다 0.999...)

#### 데이터가 균형 잡혀 있어서 예측이 단단할 수 있음
→ 그래서 전환 “경계선”에 애매하게 걸친 사람이 안 보이는 거야

#### 데이터셋 사이즈가 작거나 test셋에 그런 케이스가 없는 것일 수도 있음

In [ ]:
# 확률을 더 넓게: 0.4 ~ 0.8 사이
unstable_candidates = result_df[
    (result_df['predicted_prob'] >= 0.4) &
    (result_df['predicted_prob'] <= 0.8) &
    (result_df['real_segment'] != result_df['predicted_segment'])
]

print(unstable_candidates[['real_segment', 'predicted_segment', 'predicted_prob']].head())


### 실제 학습된 모델의 예측 정확도

In [ ]:
# 기본
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 결측치 시각화를 위한 라이브러리
import missingno

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.histplot(result_df['predicted_prob'], bins=30, kde=True)
plt.axvline(0.6, color='red', linestyle='--', label='0.6 threshold')
plt.title("📊 전체 예측 확률 분포")
plt.xlabel("Predicted Probability")
plt.ylabel("Count")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
print(y_proba[:5])

## 🔍 예측 확률 분석 결과 정리

### ✅ 예측 확률 분포 해석
- 대부분의 예측 확률이 `0.999 이상`에 몰려 있음
- 모델이 예측한 세그먼트에 대해 **거의 100% 확신을 가지고 있음**
- 따라서, 전환 가능성이 있는 **경계선 고객 (예: 0.5 ~ 0.74)** 은 거의 없음

---

### ⚠️ 문제점 요약

| 항목 | 설명 |
|------|------|
| 과확신 (Overconfidence) | 모델이 모든 예측에 대해 너무 확신함 (확률 분포가 한쪽으로 치우침) |
| 전환 후보 부족 | `real_segment ≠ predicted_segment` 이면서 `확률이 애매한 고객`이 거의 없음 |
| 해석 어려움 | SHAP 없이 예측 결과만으로는 KPI 분석이나 전환 인사이트 도출이 어려움 |

---

### 📌 원인 추정

1. **과적합**: validation data까지 외운 듯한 모델 동작
2. **클래스 불균형**: 특정 세그먼트가 많아서 그쪽으로 쏠림
3. **강한 피처**: 일부 피처가 세그먼트를 거의 결정할 정도로 강력

---

### 🎯 다음 단계 제안

- SHAP 분석으로 모델이 **왜 확신하고 있는지** 피처 기반으로 해석
- 다른 모델(Logistic, LightGBM 등)과 비교
- 예측 데이터셋을 진짜 운영 고객 데이터로 교체해보기


In [ ]:
import shap

# 1. TreeExplainer 정의
explainer = shap.Explainer(xgb_model)

# 2. validation 데이터에 대한 shap 값 계산
shap_values = explainer(X_val)

# 3. SHAP summary plot (feature 중요도 전체 시각화)
shap.summary_plot(shap_values, X_val, plot_type="bar")

### 🎯 SHAP 분석 기반 KPI 후보 지표

1. **정상청구원금_B5M**  
   - 최근 5개월 동안 청구된 정상 원금 총액  
   - 고객의 상환 규모 또는 부채 수준을 보여주는 주요 지표

2. **이용금액_R3M_신용체크**  
   - 최근 3개월간 신용 및 체크카드 이용액  
   - 카드 사용 패턴과 소비 성향 반영

3. **청구금액_R6M**  
   - 최근 6개월 누적 청구 금액  
   - 고액 청구 고객 → 상향 가능성 판단 가능

4. **카드론이용금액_누적 / 평잔_할부_3M** 등도 뒤따라 등장  
   - 리볼빙·할부 습관은 리스크 요인 or 등급 유지 요인으로 작용 가능
